In [ ]:
# 影片裁減
import ffmpeg

ffmpeg_path = r'C:\Program Files\ffmpeg\bin\ffmpeg.exe'  # 指定ffmpeg路径

def trim_and_crop_video(input_file, output_file, crop_width, crop_height, x, y):
    try:
        (
            ffmpeg
            .input(input_file)
            .filter('crop', crop_width, crop_height, x, y)
            .output(output_file, vcodec='libx264', acodec='aac', preset='veryfast', crf=18, audio_bitrate='192k', map='0:a:0')
            .run(cmd=ffmpeg_path, overwrite_output=True, capture_stdout=True, capture_stderr=True)
        )
        print(f"Trimming and cropping successful: {output_file}")
    except ffmpeg.Error as e:
        print(f"Error occurred: {e.stderr.decode('utf-8')}")
    except Exception as e:
        print(f"Unexpected error: {e}")



# 示例用法
input_file = r"C:\Users\User\Downloads\AV\抖音第二集.mp4"
output_file = r"C:\Users\User\Downloads\AV\抖音第2集.mp4"

# 原始分辨率 768x480，裁剪到 270x480
crop_width = 270  # 9:16 的宽度
crop_height = 480  # 高度保持不变
x = (768 - crop_width) // 2  # 从水平中心裁剪
y = 0  # 从顶部开始裁剪

trim_and_crop_video(input_file, output_file,  crop_width=crop_width, crop_height=crop_height, x=x, y=y)


In [ ]:
import yt_dlp as youtube_dl

def download_watch_later(cookie_file, output_path):
    try:
        ydl_opts = {
            'outtmpl': f'{output_path}/%(title)s.%(ext)s',
            'format': 'bestvideo+bestaudio/best',
            'cookies': cookie_file,  # 提供包含登录信息的 cookie 文件
            'quiet': False,  # 显示详细的下载信息
        }
        # 通过使用正确的播放列表 ID 下载
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download(['https://www.youtube.com/playlist?list=WL'])
        print(f"Downloaded Watch Later playlist to: {output_path}")
    except Exception as e:
        print(f"Error: {e}")

# 示例用法
cookie_file = r"C:\Users\User\Desktop\cookie.txt"  # 替换为包含登录信息的 cookie 文件路径
output_path = r"C:\Users\User\Downloads\watch_later"  # 保存到的文件夹
download_watch_later(cookie_file, output_path)


In [ ]:
# 下載文字檔中的所有YouTube連結影片
import re
import os
import subprocess
import requests

def extract_youtube_links(file_path):
    """從指定檔案中提取所有 YouTube 連結"""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # 正則表達式匹配所有 http/https 開頭的 YouTube 影片連結
    youtube_links = re.findall(r'(https?://(?:www\.)?(?:youtube\.com|youtu\.be)[^\s]+)', content)

    return youtube_links

def is_video_available(url):
    """檢查影片是否能夠訪問"""
    try:
        response = requests.head(url, allow_redirects=True)
        if response.status_code == 200:
            return True
    except requests.RequestException as e:
        print(f"檢查影片時出現錯誤: {str(e)}")
    return False

def download_video(link):
    """使用 yt-dlp 下載影片並處理18禁影片"""
    print(f"正在下載影片: {link}")
    
    # 設定 yt-dlp 的下載選項
    ydl_command = [
        'yt-dlp',
        '-f', 'best',  # 下載最高畫質
        '--age-limit', '18',  # 允許下載18禁影片
        link
    ]
    
    try:
        # 使用 subprocess 執行 yt-dlp 命令來下載影片
        result = subprocess.run(ydl_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if result.returncode == 0:
            print(f"影片下載成功: {link}")
        else:
            print(f"下載失敗，錯誤訊息:\n{result.stderr.decode('utf-8')}")
    
    except Exception as e:
        print(f"下載影片時發生錯誤: {str(e)}")

def download_videos(links):
    """嘗試下載所有連結，跳過無法下載的影片"""
    if not links:
        print("沒有找到 YouTube 連結。")
        return

    for link in links:
        if is_video_available(link):
            download_video(link)
        else:
            print(f"無法下載或影片無效，跳過: {link}")

if __name__ == "__main__":
    # 提供記事本文件的路徑
    file_path = 'C:/Users/User/Desktop/1.txt'  # 修改為你的記事本路徑

    # 1. 從記事本中提取所有 YouTube 連結
    youtube_links = extract_youtube_links(file_path)

    # 2. 下載所有找到的 YouTube 影片，跳過無效影片
    download_videos(youtube_links)


In [ ]:
#將影片轉成多張圖片
import cv2
import os

# 創建資料夾如果它不存在
output_folder = 'movie'
os.makedirs(output_folder, exist_ok=True)

# 打開視頻文件
vidcap = cv2.VideoCapture(r"C:\Users\User\Downloads\finish.mp4")
frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)

# 計算影片的總長度（秒）
duration = frame_count / fps

# 指定開始和結束的時間點 (秒)
start_time = 0
end_time = duration  # 到第20秒結束

# 設置視頻的當前幀位置到開始時間點
vidcap.set(cv2.CAP_PROP_POS_MSEC, start_time * 1000)

fps = int(vidcap.get(cv2.CAP_PROP_FPS))  # 獲取影片的幀率
frame_count = 0
success, image = vidcap.read()

while success:
    current_time = vidcap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # 當前幀的時間 (秒)
    if current_time > end_time:
        break

    cv2.imwrite(os.path.join(output_folder, f"frame_{frame_count:04d}.jpg"), image)  # 保存幀為 jpg 文件
    success, image = vidcap.read()  # 讀取下一幀
    frame_count += 1


In [10]:
import re
import os
import subprocess
import requests

def extract_youtube_links(file_path):
    """從指定檔案中提取所有 YouTube 連結 (包括影片和播放清單)"""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # 正則表達式匹配所有 http/https 開頭的 YouTube 影片和播放清單連結
    youtube_links = re.findall(r'(https?://(?:www\.)?(?:youtube\.com|youtu\.be)[^\s]+)', content)

    return youtube_links

def is_video_available(url):
    """檢查影片是否能夠訪問"""
    try:
        response = requests.head(url, allow_redirects=True)
        if response.status_code == 200:
            return True
    except requests.RequestException as e:
        print(f"檢查影片時出現錯誤: {str(e)}")
    return False

def download_video_or_playlist(link):
    """使用 yt-dlp 下載單個影片或整個播放清單"""
    print(f"正在下載: {link}")
    
    # 設定 yt-dlp 的下載選項
    ydl_command = [
        'yt-dlp',
        '-f', 'best',  # 下載最高畫質
        '--yes-playlist',  # 允許下載整個播放清單
        '--age-limit', '18',  # 允許下載18禁影片
        link
    ]
    
    try:
        # 使用 subprocess 執行 yt-dlp 命令來下載影片或播放清單
        result = subprocess.run(ydl_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if result.returncode == 0:
            print(f"下載成功: {link}")
        else:
            print(f"下載失敗，錯誤訊息:\n{result.stderr.decode('utf-8')}")
    
    except Exception as e:
        print(f"下載影片或播放清單時發生錯誤: {str(e)}")

def download_videos(links):
    """嘗試下載所有連結，跳過無法下載的影片"""
    if not links:
        print("沒有找到 YouTube 連結。")
        return

    for link in links:
        if is_video_available(link):
            download_video_or_playlist(link)
        else:
            print(f"無法下載或影片無效，跳過: {link}")

if __name__ == "__main__":
    # 提供記事本文件的路徑
    file_path = 'C:/Users/User/桌面/note.txt'  # 修改為你的記事本路徑

    # 1. 從記事本中提取所有 YouTube 連結（影片和播放清單）
    youtube_links = extract_youtube_links(file_path)

    # 2. 下載所有找到的 YouTube 影片或播放清單，跳過無效影片
    download_videos(youtube_links)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/User/桌面/note.txt'

In [ ]:
# 下載YouTube播放清單的所有影片
import os
import subprocess
import re
from tqdm import tqdm  # 引入 tqdm 库

# 配置参数
input_audio_folder = 'C:\\Users\\User\\Desktop\\perfect\\audio'
output_video_folder = 'C:/Users/User/Desktop/制作影片'
background_color = 'black'
video_resolution = '1280x720'  # 视频分辨率

# 确保输出文件夹存在
os.makedirs(output_video_folder, exist_ok=True)

def get_audio_duration(audio_file):
    """获取音频文件的持续时间（秒）。"""
    ffmpeg_path = r'C:\Program Files\ffmpeg\bin\ffmpeg.exe'
    print(f"Attempting to get duration for file: {audio_file}")
    try:
        result = subprocess.run(
            [ffmpeg_path, '-i', audio_file],
            stderr=subprocess.PIPE,
            text=True,  # 使用 text=True 确保输出为字符串
            encoding='utf-8'  # 强制使用 utf-8 编码处理输出
        )
        print(result.stderr)  # 打印 stderr 以查看 ffmpeg 输出
        duration_match = re.search(r'Duration: (\d+:\d+:\d+\.\d+)', result.stderr)
        if duration_match:
            duration_str = duration_match.group(1)
            h, m, s = map(float, duration_str.split(':'))
            return h * 3600 + m * 60 + s
        else:
            raise ValueError("Could not determine audio duration.")
    except Exception as e:
        print(f"Error getting duration for {audio_file}: {e}")
        raise

def process_audio_file(audio_file):
    # 获取音频文件名和扩展名
    file_name, _ = os.path.splitext(os.path.basename(audio_file))
    output_video_file = os.path.join(output_video_folder, f"{file_name}.mp4")
    
    # 打印音频文件路径用于调试
    print(f'Processing file: {audio_file}')
    
    # 获取音频持续时间
    try:
        audio_duration = get_audio_duration(audio_file)
    except Exception as e:
        print(f"Error processing file {audio_file}: {e}")
        return
    
    # 使用 FFmpeg 生成黑色背景视频，并将音频添加到视频中
    ffmpeg_path = r'C:\Program Files\ffmpeg\bin\ffmpeg.exe'
    cmd = [
        ffmpeg_path,
        '-f', 'lavfi', 
        '-i', f'color=c={background_color}:s={video_resolution}:d={audio_duration}',  # 生成黑色背景
        '-i', audio_file,  # 输入音频文件
        '-c:v', 'libx264',
        '-preset', 'medium',
        '-crf', '30',  # 降低视频质量以提高处理速度
        '-c:a', 'aac',
        '-b:a', '192k',  # 提高音质
        '-shortest',  # 确保视频时长与音频一致
        output_video_file
    ]
    
    # 执行命令
    try:
        subprocess.run(cmd, check=True)
        print(f'Processed {audio_file} -> {output_video_file}')
        
        # 删除原始音频文件
        os.remove(audio_file)
        print(f'Deleted original file: {audio_file}')
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg error processing file {audio_file}: {e}")

def batch_process_audio_files(input_folder):
    """批量处理文件夹中的音频文件。"""
    audio_files = [os.path.join(input_folder, file_name) for file_name in os.listdir(input_folder)
                   if file_name.lower().endswith(('.aac', '.m4a', '.wav', '.mp3'))]  # 处理常见音频格式

    # 使用 tqdm 显示进度条
    for audio_file in tqdm(audio_files, desc="Processing audio files"):
        process_audio_file(audio_file)

# 批量处理音频文件
batch_process_audio_files(input_audio_folder)


ffmpeg -loop 1 -i "C:/Users/User/Pictures/Screenshots/螢幕擷取畫面 2024-05-15 145545.png" -i "D:/錄音備份_已死筆記本/1月1日 下午10點38分.aac" -c:v libx264 -preset medium -crf 22 -c:a aac -b:a 192k -shortest -t 20 "C:/Users/User/Desktop/output.mp4"     
